<a href="https://colab.research.google.com/github/dinagalevska/NLP/blob/master/NLP_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers

In [2]:
import pandas as pd
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import classification_report
import torch
from tqdm import tqdm
import time

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Excercise 1

In [4]:
model_name = 't5-base'
t5_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
t5_tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
train_data = pd.read_csv('train_en.txt', sep='\t')
val_data = pd.read_csv('val_en.txt', sep='\t')
test_data = pd.read_csv('test_en.txt', sep='\t')

In [25]:
def evaluate_offensive_text_detection(data, model, tokenizer, show_progress_bar=False, batch_size=16, max_length=20):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    sentences = data['Sentence'].tolist()
    labels = data['Style'].apply(lambda x: 1 if x == 'toxic' else 0).tolist()
    predictions = []

    total_sentences = len(sentences)

    progress = tqdm(range(0, total_sentences, batch_size), desc="Processing") if show_progress_bar else range(0, total_sentences, batch_size)

    for i in progress:
        batch_sentences = sentences[i:i + batch_size]
        prompts = [f"Classify the following sentence into toxic or neutral: {sentence}" for sentence in batch_sentences]

        tokens = tokenizer(prompts, return_tensors='pt', padding=True, truncation=True).to(device)

        with torch.no_grad():
            output = model.generate(tokens.input_ids, max_length=max_length)

        results = tokenizer.batch_decode(output, skip_special_tokens=True)
        predictions.extend([1 if 'toxic' in result.lower() else 0 for result in results])

        if show_progress_bar:
            progress.set_postfix({"Processed": min(i + batch_size, total_sentences)})

    report = classification_report(labels, predictions, target_names=["neutral", "toxic"])
    return report

In [26]:
train_report = evaluate_offensive_text_detection(train_data, t5_model, t5_tokenizer, show_progress_bar=True)
print("Train Evaluation Report:")
print(train_report)

Processing: 100%|██████████| 1565/1565 [11:40<00:00,  2.23it/s, Processed=25040]

Train Evaluation Report:
              precision    recall  f1-score   support

     neutral       0.48      0.83      0.61     12520
       toxic       0.39      0.11      0.17     12520

    accuracy                           0.47     25040
   macro avg       0.44      0.47      0.39     25040
weighted avg       0.44      0.47      0.39     25040



In [27]:
val_report = evaluate_offensive_text_detection(val_data, t5_model, t5_tokenizer, show_progress_bar=True)
print("Validation Evaluation Report:")
print(val_report)

Processing: 100%|██████████| 196/196 [01:42<00:00,  1.91it/s, Processed=3130]

Validation Evaluation Report:
              precision    recall  f1-score   support

     neutral       0.48      0.83      0.61      1565
       toxic       0.38      0.10      0.16      1565

    accuracy                           0.47      3130
   macro avg       0.43      0.47      0.39      3130
weighted avg       0.43      0.47      0.39      3130



In [29]:
test_report = evaluate_offensive_text_detection(test_data, t5_model, t5_tokenizer, show_progress_bar=True)
print("Test Evaluation Report:")
print(test_report)

Processing: 100%|██████████| 196/196 [01:35<00:00,  2.05it/s, Processed=3132]

Test Evaluation Report:
              precision    recall  f1-score   support

     neutral       0.48      0.83      0.61      1566
       toxic       0.37      0.10      0.16      1566

    accuracy                           0.46      3132
   macro avg       0.42      0.46      0.38      3132
weighted avg       0.42      0.46      0.38      3132



#Excercise 2

In [56]:
def evaluate_few_shot(data, model, tokenizer, n_samples, prompt_template, show_progress_bar=False, max_length=20):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    sentences = data['Sentence'].tolist()
    labels = data['Style'].apply(lambda x: 1 if x == 'toxic' else 0).tolist()
    print(set(labels))
    predictions = []

    few_shot_examples = [("This is a toxic text.", 1), ("This is a neutral text.", 0)]

    few_shot_prompts = [
        f"{prompt_template.replace('<text>', text).replace('<label>', 'toxic' if label == 1 else 'neutral')}"
        for text, label in few_shot_examples[:n_samples]
    ]

    progress = tqdm(range(len(sentences)), desc="Processing") if show_progress_bar else range(len(sentences))

    for i in progress:
        prompt = f"{' '.join(few_shot_prompts)} Classify the following text: {sentences[i]} into toxic or neutral."

        tokens = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to(device)

        with torch.no_grad():
            output = model.generate(tokens.input_ids, max_length=max_length)

        result = tokenizer.decode(output[0], skip_special_tokens=True)
        predictions.append(1 if 'toxic' in result.lower() else 0)

    report = classification_report(labels, predictions, target_names=["neutral", "toxic"], zero_division=1)
    return report


In [37]:
n_samples_values = [5, 10]
prompts = [
    "Here is a text: <text>, which is <label>. Classify the following text: <text> into toxic or neutral.",
    "Here is a text: <text>, which is not <label>. Classify the following text: <text> into toxic or neutral."
]

In [61]:
for prompt in prompts:
    for n in n_samples_values:
        print(f"Evaluating for n_samples={n} with prompt:\n{prompt}\n")
        report = evaluate_few_shot(train_data[:100], t5_model, t5_tokenizer, n, prompt, show_progress_bar=True)
        print(f"Classification Report for n_samples={n}:\n{report}\n")

Evaluating for n_samples=5 with prompt:
Here is a text: <text>, which is <label>. Classify the following text: <text> into toxic or neutral.

{1}


Processing: 100%|██████████| 100/100 [00:44<00:00,  2.25it/s]


Classification Report for n_samples=5:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.89      0.94       100

    accuracy                           0.89       100
   macro avg       0.50      0.95      0.47       100
weighted avg       1.00      0.89      0.94       100


Evaluating for n_samples=10 with prompt:
Here is a text: <text>, which is <label>. Classify the following text: <text> into toxic or neutral.

{1}


Processing: 100%|██████████| 100/100 [00:33<00:00,  2.97it/s]


Classification Report for n_samples=10:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.89      0.94       100

    accuracy                           0.89       100
   macro avg       0.50      0.95      0.47       100
weighted avg       1.00      0.89      0.94       100


Evaluating for n_samples=5 with prompt:
Here is a text: <text>, which is not <label>. Classify the following text: <text> into toxic or neutral.

{1}


Processing: 100%|██████████| 100/100 [00:28<00:00,  3.47it/s]


Classification Report for n_samples=5:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.85      0.92       100

    accuracy                           0.85       100
   macro avg       0.50      0.93      0.46       100
weighted avg       1.00      0.85      0.92       100


Evaluating for n_samples=10 with prompt:
Here is a text: <text>, which is not <label>. Classify the following text: <text> into toxic or neutral.

{1}


Processing: 100%|██████████| 100/100 [00:28<00:00,  3.52it/s]

Classification Report for n_samples=10:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.85      0.92       100

    accuracy                           0.85       100
   macro avg       0.50      0.93      0.46       100
weighted avg       1.00      0.85      0.92       100




морав дел од датата бидејќи многу време беше потребно(затоа ставив и zero_division)

само со 5,10 за н бидејќи многу време е потребно

затоа и резултатите се лоши бидејки не ја користам цела дата

In [33]:
for prompt in prompts:
    for n in n_samples_values:
        print(f"Evaluating for n_samples={n} with prompt:\n{prompt}\n")
        report = evaluate_few_shot(val_data[:90], t5_model, t5_tokenizer, n, prompt, show_progress_bar=True)
        print(f"Classification Report for n_samples={n}:\n{report}\n")

Evaluating for n_samples=5 with prompt:
Here is a text: <text>, which is <label>. Classify the following text: <text> into toxic or neutral.



Processing: 100%|██████████| 90/90 [00:35<00:00,  2.53it/s]


Classification Report for n_samples=5:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.81      0.90        90

    accuracy                           0.81        90
   macro avg       0.50      0.91      0.45        90
weighted avg       1.00      0.81      0.90        90


Evaluating for n_samples=10 with prompt:
Here is a text: <text>, which is <label>. Classify the following text: <text> into toxic or neutral.



Processing: 100%|██████████| 90/90 [00:27<00:00,  3.25it/s]


Classification Report for n_samples=10:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.81      0.90        90

    accuracy                           0.81        90
   macro avg       0.50      0.91      0.45        90
weighted avg       1.00      0.81      0.90        90


Evaluating for n_samples=5 with prompt:
Here is a text: <text>, which is not <label>. Classify the following text: <text> into toxic or neutral.



Processing: 100%|██████████| 90/90 [00:26<00:00,  3.45it/s]


Classification Report for n_samples=5:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.78      0.88        90

    accuracy                           0.78        90
   macro avg       0.50      0.89      0.44        90
weighted avg       1.00      0.78      0.88        90


Evaluating for n_samples=10 with prompt:
Here is a text: <text>, which is not <label>. Classify the following text: <text> into toxic or neutral.



Processing: 100%|██████████| 90/90 [00:25<00:00,  3.48it/s]

Classification Report for n_samples=10:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.78      0.88        90

    accuracy                           0.78        90
   macro avg       0.50      0.89      0.44        90
weighted avg       1.00      0.78      0.88        90




In [62]:
for prompt in prompts:
    for n in n_samples_values:
        print(f"Evaluating for n_samples={n} with prompt:\n{prompt}\n")
        report = evaluate_few_shot(test_data[:80], t5_model, t5_tokenizer, n, prompt, show_progress_bar=True)
        print(f"Classification Report for n_samples={n}:\n{report}\n")

Evaluating for n_samples=5 with prompt:
Here is a text: <text>, which is <label>. Classify the following text: <text> into toxic or neutral.

{1}


Processing: 100%|██████████| 80/80 [00:26<00:00,  3.03it/s]


Classification Report for n_samples=5:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.88      0.93        80

    accuracy                           0.88        80
   macro avg       0.50      0.94      0.47        80
weighted avg       1.00      0.88      0.93        80


Evaluating for n_samples=10 with prompt:
Here is a text: <text>, which is <label>. Classify the following text: <text> into toxic or neutral.

{1}


Processing: 100%|██████████| 80/80 [00:25<00:00,  3.18it/s]


Classification Report for n_samples=10:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.88      0.93        80

    accuracy                           0.88        80
   macro avg       0.50      0.94      0.47        80
weighted avg       1.00      0.88      0.93        80


Evaluating for n_samples=5 with prompt:
Here is a text: <text>, which is not <label>. Classify the following text: <text> into toxic or neutral.

{1}


Processing: 100%|██████████| 80/80 [00:31<00:00,  2.57it/s]


Classification Report for n_samples=5:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.85      0.92        80

    accuracy                           0.85        80
   macro avg       0.50      0.93      0.46        80
weighted avg       1.00      0.85      0.92        80


Evaluating for n_samples=10 with prompt:
Here is a text: <text>, which is not <label>. Classify the following text: <text> into toxic or neutral.

{1}


Processing: 100%|██████████| 80/80 [00:24<00:00,  3.28it/s]

Classification Report for n_samples=10:
              precision    recall  f1-score   support

     neutral       0.00      1.00      0.00         0
       toxic       1.00      0.85      0.92        80

    accuracy                           0.85        80
   macro avg       0.50      0.93      0.46        80
weighted avg       1.00      0.85      0.92        80


